In [1]:
#================================================================
# Yolo V-3 by irfanhasib.me@hgmail.com
# Inspired by -
# GitHub      : https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3
#================================================================
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED']=str(0)
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import time
import glob
from matplotlib import pyplot as plt
from multiprocessing import Process, Queue, Pipe
import time
import shutil
import json
from tqdm import tqdm,trange
import pickle
import zlib
from datetime import datetime

In [2]:
from yolo.model import YoloModel, calc_yolo_loss, calc_seg_loss
from yolo.decoder import YoloDecodeNetout
from yolo.dataset import Dataset
from yolo.eval import get_mAP
from yolo.utils import Utils
from yolo.seg_loader import Seg_Utils
from yolo.config import *
from yolo.tf import *

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
yolo_test= False
yolo_eval= False
seg_test= False
sanity_check = False
data_gen = False
debug = False

In [4]:
if yolo_test == True or yolo_eval== True or seg_test==True or sanity_check== True or debug == True:
    save_notebook = False
else:
    save_notebook = True
    
if save_notebook == True:
    if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER): os.makedirs(TRAIN_CHECKPOINTS_FOLDER)
    with open(TRAIN_CHECKPOINTS_FOLDER +'/params.txt','w') as file:
        log_str='Time '.ljust(30)+': '+str(datetime.now())+'\n'
        for key in list(params.keys())[:-1]:
            if key[:2] != '__':
                log_str += key.ljust(30)+': ' + str(params[key])+'\n'
        print(log_str)
        file.write(log_str)

Time                          : 2021-08-12 18:00:43.089144
TRAIN_CHECKPOINTS_FOLDER      : logs/exp-401
YOLO_TYPE                     : yolov3
YOLO_MODEL                    : LITE
YOLO_FRAMEWORK                : tf
YOLO_V3_WEIGHTS               : model_data/yolov3.weights
YOLO_V4_WEIGHTS               : model_data/yolov4.weights
YOLO_V3_TINY_WEIGHTS          : model_data/yolov3-tiny.weights
YOLO_V4_TINY_WEIGHTS          : model_data/yolov4-tiny.weights
YOLO_TRT_QUANTIZE_MODE        : INT8
YOLO_CUSTOM_WEIGHTS           : True
YOLO_COCO_CLASSES             : model_data/coco/coco.names
YOLO_STRIDES                  : [16, 32, 64]
YOLO_IOU_LOSS_THRESH          : 0.5
YOLO_ANCHOR_PER_SCALE         : 3
YOLO_MAX_BBOX_PER_SCALE       : 100
YOLO_INPUT_SIZE               : 256
YOLO_ANCHORS                  : [[[10, 14], [23, 27], [37, 58]], [[81, 82], [135, 169], [344, 319]], [[0, 0], [0, 0], [0, 0]]]
SEED                          : 0
TRAIN_YOLO_TINY               : True
TRAIN_MODEL_SCALE        

In [ ]:
if save_notebook == True:
    curr_time=time.time()
    print('System time : ',curr_time)
    %autosave 1
    time.sleep(3)
    
    shutil.copy('yolo-v3.ipynb',TRAIN_CHECKPOINTS_FOLDER+'/yolo_v3_log_'+str(curr_time)+'.ipynb')
    if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER+'/yolo/'): os.makedirs(TRAIN_CHECKPOINTS_FOLDER+'/yolo/')
    files = glob.glob('yolo/*')
    for file in files:
        try : shutil.copy(file,TRAIN_CHECKPOINTS_FOLDER+'/'+file)
        except PermissionError:
            print('PermissionError : ',file)
    %autosave 120

System time :  1628758843.1071465


Autosaving every 1 seconds


In [ ]:
if yolo_test == True:
    if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER+'/pred_imgs'): os.makedirs(TRAIN_CHECKPOINTS_FOLDER+'/pred_imgs')
    #video_path   = "./IMAGES/test.mp4"
    img_path   = "D:/COCO/val2017/"
    yolo = YoloModel()
    yolo_model=yolo.get_model()
    decoder = YoloDecodeNetout()

    #decoder.detect_video(yolo_model, video_path, input_size=288, show=True, score_threshold=0.1, iou_threshold=0.2, rectangle_colors='')
    decoder.detect_images(yolo_model, img_path, output_path=TRAIN_CHECKPOINTS_FOLDER+'/pred_imgs/',input_size=256, show=True, score_threshold=0.3, iou_threshold=0.5, rectangle_colors='')

In [ ]:
if yolo_eval == True:
    res_dict=[]
    for min_overlap in list(range(50,100,5)):
        min_overlap = min_overlap/100
        for iou_threshold in [0.1]:#,0.2,0.3,0.3,0.4,0.5]:#[0.1,0.2,0.3,0.4,0.5]:
            for score_threshold in [0.0]:#,0.05,0.1,0.2]:#[0.1,0.2,0.3,0.4,0.5]:
                yolo = YoloModel()
                yolo_model=yolo.get_model()
                decoder = YoloDecodeNetout()

                testset = Dataset('test')
                out=get_mAP(yolo_model, testset, decoder, min_overlap= min_overlap ,score_threshold=score_threshold, iou_threshold=iou_threshold, TEST_INPUT_SIZE=YOLO_INPUT_SIZE)
                res_dict+=[[out,min_overlap,score_threshold,iou_threshold]]
                print(res_dict)
    
    with open(TRAIN_CHECKPOINTS_FOLDER+'/scores_0.1_0.0001.pkl','wb') as file:
        pickle.dump(res_dict,file)
    print(sum([res[0] for res in res_dict])/10)

In [ ]:
if seg_test == True:
    trainset = Dataset('train')
    testset = Dataset('test')
    
    
    yolo = YoloModel()
    yolo_model=yolo.get_model()
        
    for image , label in trainset:
        break
        
    out = yolo_model.predict(image)
    plt.imshow(label[3][1].max(axis=-1))
    plt.show()
    plt.imshow(out[2][1][:,:,:len(CLASS_NAMES)].max(axis=-1))
    plt.show()

    for image , label in testset:
        break
    out = yolo_model.predict(image)
    plt.imshow(label[3][1].max(axis=-1))
    plt.show()
    plt.imshow(out[2][1][:,:,:len(CLASS_NAMES)].max(axis=-1))
    plt.show()
    
    #plt.imshow(label[3][1])
    #plt.show()
    #plt.imshow(out[2][0][:,:].max(axis=-1))
    #plt.imshow(image[0])
    #plt.show()

In [ ]:
#12.05867419755847
#12.119057160200935

Model=tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=80
)
Model.summary()

In [ ]:
if sanity_check == True:
    trainset = Dataset('train')
    testset  = Dataset('test')
    for train_img, train_label in trainset:
        print('..')
        break
    for test_img, test_label in testset:
        print('...')
        break
    decoder = YoloDecodeNetout()
    pred_bbox = [label[0][0] for label in train_label]#[label_sbbox, label_mbbox, label_lbbox]
    
    pred_bbox = [np.reshape(x, (-1, np.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = np.concatenate(pred_bbox, axis=0)

    bboxes = decoder.decode_boxes(pred_bbox, train_img[0], YOLO_INPUT_SIZE, TEST_SCORE_THRESHOLD)
    bboxes = decoder.nms(bboxes, TEST_IOU_THRESHOLD, method='nms')

    out=Utils.draw_bbox(train_img[0], bboxes, conf=True,show_label=True, show_confidence = True, Text_colors=(255,255,0), rectangle_colors='', tracking=False)
    plt.imshow(train_img[0])
    plt.show()
    plt.imshow(out)

In [ ]:
wts_check=False
if wts_check== True:
    wts = yolo_model.trainable_weights
    for i in range(len(wts)):
        if len(wts[i].shape) and wts[i].shape[0]==3: 
            print(wts[i].shape)
            _wts = tf.abs(wts[i])
            vector = tf.reduce_sum(_wts,axis=[0,1])
            norm_rev_vec = 1 - vector/tf.reduce_max(vector)
            plt.imshow(norm_rev_vec,cmap='gray')
            print(tf.reduce_mean(norm_rev_vec))
            plt.show()

In [ ]:
class loss_dict_obj(dict):
        def tf2np(self,val):
            if hasattr(val,'numpy'):
                val=val.numpy()
            else:
                if val==None: val=0
            return val

        def sum_update(self,c_dict):
            for key,val in c_dict.items():
                    if key in list(self.keys()):
                        self[key]+=self.tf2np(c_dict[key])
                    else:
                        self[key]=self.tf2np(c_dict[key])


        def ext_update(self,c_dict,_ext='_ext'):
            for key,val in c_dict.items():
                     self[_ext+key]=self.tf2np(val)

        def divide(self,div_val):
            for key,val in self.items():
                if type(div_val)==dict or type(div_val)==loss_dict_obj:
                    self[key]/=div_val[key]
                else:
                    self[key]/=div_val

        def _sum(self):
            total=0
            for val in self.values():
                total+=val
            return total

        def copy_keys(self,c_dict,keys):
            for key in keys:
                self[key]=c_dict[key]

        def apply(self,func):
            for key in self.keys():
                self[key]=self.tf2np(func(self[key]))

def save_loss_logs(loss_dict,epoch):
    if epoch==0: log_str=','.join(list(loss_dict.keys()))+'\n'
    else : log_str = ''
    log_str += ','.join(list(map(str,loss_dict.values())))+'\n'

    with open(os.path.join(TRAIN_CHECKPOINTS_FOLDER,'loss.csv'),'a+') as file:
        file.write(log_str)
            
def save_std_logs(train_loss,all_logs,epoch):
    def str_func(x):
        return str(x.numpy())
    
    if epoch==0 : log_str='epoch,'+','.join(list(train_loss.keys()))+'\n'
    else : log_str = ''
            
    for _ind in range(no_train_batch): 
        log_str += str(epoch)+','
        log_str += ','.join(list(map(str_func,all_logs[_ind])))+'\n'

    with open(os.path.join(TRAIN_CHECKPOINTS_FOLDER,'all_loss.csv'),'a+') as file:
        file.write(log_str)
        
def save_sample_losses(sample_losses,epoch):
    
    for _lind,loss_name in zip([0,1],['det','seg']):
        if TRAIN_LOSS_WTS[_lind]:
            if epoch==0 : log_str='epoch,'+','.join(list(map(str,range(TRAIN_BATCH_SIZE))))+'\n'
            else : log_str = ''
            for _ind in range(no_train_batch): 
                log_str += str(epoch)+','
                log_str += ','.join(list(map(str,sample_losses[_ind][_lind].numpy())))+'\n'

            with open(os.path.join(TRAIN_CHECKPOINTS_FOLDER,'sample_loss_'+loss_name+'.csv'),'a+') as file:
                file.write(log_str)
            
@tf.function#(experimental_relax_shapes=True)  
def train_step(image_data, target,epoch):
        conv_layer_ind = [0,3,6,9,12,15,18]
        train_loss_dict = loss_dict_obj()
        sample_loss_dict= loss_dict_obj()
        vec_dict = {ind: 1.0 for ind in conv_layer_ind}
        det_loss = seg_loss = kl_coef = 0.0
        gradients1 = gradients2 = [None]*len(yolo_model.trainable_variables)
        
        with tf.GradientTape(persistent=True) as tape:
            pred_result = yolo_model(image_data)
            del image_data
            
            if TRAIN_LOSS_WTS[0]:
                grid = 3 if not TRAIN_YOLO_TINY else 2
                for i in range(grid):
                    conv, pred = pred_result[i*2], pred_result[i*2+1]
                    loss_dict, smp_loss_dict, grid_det_loss = calc_yolo_loss(pred, conv, *target[i], i)
                    train_loss_dict.sum_update(loss_dict)
                    sample_loss_dict.sum_update(smp_loss_dict)
                    det_loss += grid_det_loss
                gradients1 = tape.gradient(TRAIN_LOSS_WTS[0] * det_loss, yolo_model.trainable_variables)
                    
            if TRAIN_USE_SEG or TRAIN_USE_SEG_BIN:
                seg_pred     = pred_result[2*(grid -1)+2:]
                seg_label    = target[3:]
                seg_loss_dict, seg_sample_losses, seg_loss = calc_seg_loss(seg_label,seg_pred)
                train_loss_dict.update(seg_loss_dict)
                sample_loss_dict.update(seg_sample_losses)
                
                
                norm_det_sample_losses = sample_loss_dict['det'] / tf.math.reduce_max(sample_loss_dict['det'])
                norm_seg_sample_losses = sample_loss_dict['det'] / tf.math.reduce_max(sample_loss_dict['seg'])
                #csim = -tf.keras.losses.CosineSimilarity()(norm_det_sample_losses,norm_seg_sample_losses)
                kl = tf.keras.losses.KLDivergence()(norm_det_sample_losses,norm_seg_sample_losses)
                kl_coef = tf.clip_by_value(1.0 - kl/100.0, 0.0, 1.0)
                train_loss_dict['kl_coef'] = kl_coef#.numpy()
                
                gradients2 = tape.gradient(TRAIN_LOSS_WTS[1] * seg_loss, yolo_model.trainable_variables)
            del pred_result,target
            
            if epoch > 0:
                wts = yolo_model.trainable_variables
                for i in conv_layer_ind:
                        _wts = tf.abs(wts[i])
                        vector = tf.reduce_sum(_wts,axis=[0,1])
                        norm_rev_vec = 1 - vector/tf.reduce_max(vector)
                        vec_dict[i] = norm_rev_vec
                del wts, _wts
                      
            
            for ind in range(len(gradients1)):
                if type(gradients1[ind]) == type(None) : gradients1[ind] = 0.0
                if type(gradients2[ind]) == type(None) : gradients2[ind] = 0.0
                    
                if ind in vec_dict.keys():
                    gradients1[ind] = vec_dict[ind] * gradients1[ind] + gradients2[ind] * vec_dict[ind]
                else:
                    gradients1[ind] = gradients1[ind] + gradients2[ind]
                    
            optimizer.apply_gradients(zip(gradients1, yolo_model.trainable_variables))
            
            '''
            global_steps.assign_add(1)
            if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
                lr = global_steps / warmup_steps * TRAIN_LR_INIT
            else:
                lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                    (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
            optimizer.lr.assign(lr.numpy())
            '''
        return train_loss_dict, sample_loss_dict
    
def add_ext(loss_dict,_ext='val_'):
    _loss_dict = {}
    for key in loss_dict.keys():
          _loss_dict[_ext+key]=loss_dict[key]
    return _loss_dict

@tf.function#(experimental_relax_shapes=True)
def validate_step(image_data, target):
    val_loss_dict = loss_dict_obj()
    with tf.GradientTape() as tape:
        pred_result = yolo_model(image_data)
        del image_data
        
        grid = 3 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_dict, _, _ = calc_yolo_loss(pred, conv, *target[i], i)
            loss_dict = add_ext(loss_dict,_ext='val_')
            val_loss_dict.sum_update(loss_dict)
        
        if TRAIN_USE_SEG or TRAIN_USE_SEG_BIN:
            seg_pred     = pred_result[2*(grid -1)+2:]
            seg_label    = target[3:]
            
            loss_dict, _, _  = calc_seg_loss(seg_label,seg_pred)
            loss_dict = add_ext(loss_dict,_ext='val_')
            val_loss_dict.sum_update(loss_dict)
            
        del pred_result, target
        
    return val_loss_dict

In [ ]:
trainset = Dataset('train')
testset = Dataset('test')
seg_utils = Seg_Utils()
steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

yolo = YoloModel(training=True)
yolo_model =yolo.get_model()
optimizer = tf.keras.optimizers.Adam(lr = TRAIN_LR)
best_val_loss = 10e8 # should be large at start
no_train_batch = trainset.num_batchs
no_val_batch  = testset.num_batchs

In [ ]:
if data_gen == True:
    for ind,[train_img, train_label] in tqdm(enumerate(trainset)):
        path=TRAIN_DATA_SAVE_PATH+'batch_{}.npy'.format(str(ind))
        np.save(path,[np.uint8(train_img*255.0),train_label[:3]]) 
        if TRAIN_USE_SEG:
            path_seg=TRAIN_DATA_SAVE_PATH+'batch_seg_{}.npy'.format(str(ind))
            train_label[3]  = tf.image.resize(train_label[3] ,(TRAIN_INPUT_SIZE//TRAIN_SEG_SCALE,TRAIN_INPUT_SIZE//TRAIN_SEG_SCALE),tf.image.ResizeMethod.BILINEAR).numpy()
            no_obj = 1 - np.clip(train_label[3].max(axis=-1)[:,:,:,np.newaxis],0,1)
            train_label[3] = np.concatenate([train_label[3],no_obj],axis = -1)
            train_label[3] = np.argmax(train_label[3],axis = -1)
            train_label[3] = np.array(train_label[3],dtype=np.uint8)
            #train_label[3] = zlib.compress(train_label[3])
            np.save(path_seg,train_label[3]) 
        #break
    for ind,[test_img, test_label] in tqdm(enumerate(testset)):
        path=TEST_DATA_SAVE_PATH+'batch_{}.npy'.format(str(ind))
        np.save(path,[np.uint8(test_img*255.0),test_label[:3]])
        if TRAIN_USE_SEG:
            path_seg=TEST_DATA_SAVE_PATH+'batch_seg_{}.npy'.format(str(ind))
            test_label[3] = tf.image.resize(test_label[3],(TEST_INPUT_SIZE//TRAIN_SEG_SCALE,TEST_INPUT_SIZE//TRAIN_SEG_SCALE),tf.image.ResizeMethod.BILINEAR).numpy()
            no_obj = 1 - np.clip(test_label[3].max(axis=-1)[:,:,:,np.newaxis],0,1)
            test_label[3] = np.concatenate([test_label[3],no_obj],axis = -1)
            test_label[3] = np.argmax(test_label[3],axis = -1)
            test_label[3] = np.array(test_label[3],dtype=np.uint8)
            #test_label[3] = zlib.compress(test_label[3])
            np.save(path_seg,test_label[3])
        #break
    del trainset, testset

In [ ]:
#pp = tf.one_hot(tr_l[0][0],80)
#plt.imshow(pp[:,:,0])

In [ ]:
yolo_model.summary()

In [ ]:
if debug == True:
    no_train_batch = 3
    no_val_batch   = 3
    TRAIN_EPOCHS   = 100

In [ ]:
if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER): os.makedirs(TRAIN_CHECKPOINTS_FOLDER)
for epoch in range(TRAIN_EPOCHS):
    all_logs = [] ; sample_losses =[]
    train_loss_dict=loss_dict_obj(); val_loss_dict = loss_dict_obj(); loss_dict={'Epoch' : epoch}
    #for image_data, target in tqdm(trainset):
    batch_seg=[]
    for batch_ind in tqdm(range(no_train_batch)):
        path=TRAIN_DATA_SAVE_PATH+'batch_{}.npy'.format(str(batch_ind))
        batch_data=np.load(path,allow_pickle=True)
        if TRAIN_USE_SEG or TRAIN_USE_SEG_BIN:
            path=TRAIN_DATA_SAVE_PATH+'batch_seg_{}.npy'.format(str(batch_ind))
            batch_seg=np.load(path,allow_pickle=True)
            batch_dt= np.uint8(255*(batch_seg!=80))
            for i in range(len(batch_dt)):
                batch_dt[i] = cv2.distanceTransform(batch_dt[i],cv2.DIST_L2,3)
            batch_dt = np.float32(batch_dt/ np.sqrt(128**2 + 128**2))
            
        image_data, target = np.float32(batch_data[0]/255.0),list(batch_data[1])+[batch_seg,batch_dt]
        train_loss , smp_loss = train_step(image_data, target,epoch)
        del image_data, target
        train_loss_dict.sum_update(train_loss)
        all_logs+=[list(train_loss.values())]
        
        samp_l = []
        for wt,loss_name in zip(TRAIN_LOSS_WTS,['det','seg']):
            if wt : samp_l +=[smp_loss[loss_name]]
        sample_losses +=[samp_l]
        
    train_loss_dict.divide(no_train_batch)
    
    iou_val, conf_val, prob_val, total_val = 0, 0, 0, 0
    batch_seg=[]
    #for image_data, target in testset:
    for batch_ind in tqdm(range(no_val_batch)):
        path=TEST_DATA_SAVE_PATH+'batch_{}.npy'.format(str(batch_ind))
        batch_data=np.load(path,allow_pickle=True)
        if TRAIN_USE_SEG or TRAIN_USE_SEG_BIN:
            path=TEST_DATA_SAVE_PATH+'batch_seg_{}.npy'.format(str(batch_ind))
            batch_seg=np.load(path,allow_pickle=True)
            batch_dt= np.uint8(255*(batch_seg!=80))
            for i in range(len(batch_dt)):
                batch_dt[i] = cv2.distanceTransform(batch_dt[i],cv2.DIST_L2,3)
            batch_dt = np.float32(batch_dt/ np.sqrt(128**2 + 128**2))
            
        image_data, target = np.float32(batch_data[0]/255.0),list(batch_data[1])+[batch_seg,batch_dt]
        val_loss = validate_step(image_data, target)
        del image_data, target
        val_loss_dict.sum_update(val_loss)
        
    val_loss_dict.divide(no_val_batch)
    loss_dict.update(train_loss_dict)
    loss_dict.update(val_loss_dict)
        
    print(loss_dict)
    
    if epoch % TRAIN_SAVE_WEIGHTS_EVERY == 0:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, 'model'+"_epoch_{}_val_det_loss_{}".format(epoch,round(loss_dict['val_det_loss'],4)))
        yolo_model.save_weights(save_directory+'/weights')
    
    if TRAIN_SAVE_BEST_ONLY and best_val_loss>loss_dict['val_det_loss']:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, 'model')
        yolo_model.save_weights(save_directory+'/weights')
        best_val_loss = loss_dict['val_det_loss']
       
    save_loss_logs(loss_dict,epoch)
    save_std_logs(train_loss,all_logs,epoch)
    save_sample_losses(sample_losses,epoch)
    

In [ ]:
#git remote add upstream https://github.com/user/repo
#git pull upstream master